In [1]:
import sqlite3
import pandas as pd
import pymysql
import re
from kss import kss
from konlpy.tag import Mecab
import pickle

In [2]:
mecab = Mecab()
ref_dic = pd.read_csv('ref_dic.csv')#ref_dic.reset_index(drop=True)

In [3]:
con1 = sqlite3.connect('ensol_news_db.sqlite')
cur1 = con1.cursor()
cur1.execute("select * from mobility")
db_tab = pd.DataFrame(cur1.fetchall())
db_tab = db_tab.iloc[:,[1,2,4,5]].drop_duplicates()

In [3]:
def _call_db_info():
    return pymysql.connect(
        host = 'trend.cb7jqghocrtb.ap-northeast-2.rds.amazonaws.com',
        port= 3306,
        user = 'root',
        password='ensol2020!',
        db = 'trend',
        charset = 'utf8')


conn = _call_db_info()
curs = conn.cursor()
tmp_insert_sql = "select * from content where date >= 20210701"
curs.execute(tmp_insert_sql)        
tmp_rslt = pd.DataFrame(curs.fetchall())
conn.commit()
conn.close()

news_tab = tmp_rslt.iloc[:,1:].drop_duplicates()

In [4]:
# Train
#print(db_tab.shape, news_tab.shape)
#tmp_article= pd.concat((db_tab.iloc[:,3], news_tab.iloc[:,3]))
# Test
print(news_tab.shape)
tmp_article= news_tab.iloc[:,3]

tmp_article = tmp_article.drop_duplicates()
tmp_article = tmp_article.reset_index(drop=True)
print(tmp_article.shape)

In [8]:
def extract_parenthese(str):
    items_lst = re.findall('\(([^)]+)', str) #extracts string in () 
    newList = [x for x in items_lst if len(x)>=2] # more than 2
    return newList

def extract_quotes(str):
    items_lst = re.findall('"([^"]*)"', str)
    return items_lst

def parentheses_(tmp_input_sent):    
    tmp_input_sent = re.sub(pattern='\(+', repl=' ', string=tmp_input_sent)#tmp_input_sent = re.sub(pattern='\(\(', repl='\(', string=tmp_input_sent)
    tmp_input_sent = re.sub(pattern='\)+', repl=' ', string=tmp_input_sent)#tmp_input_sent = re.sub(pattern='\)\)', repl='\)', string=tmp_input_sent)
    tmp_input_sent = re.sub(pattern=' +', repl=' ', string=tmp_input_sent)
    input_sent = re.sub(pattern='\\\\',   repl='', string=tmp_input_sent)
    return input_sent
    '''
    tmp_sent1, tmp_sent2 = [], []    
    s_re = re.compile('\(')#tmp_sentence[25])#.match('\(')
    e_re = re.compile('\)')
    s_m = [(m.start(0), m.end(0)) for m in s_re.finditer(input_sent)]#tmp_sentence[25])]
    e_m = [(m.start(0), m.end(0)) for m in e_re.finditer(input_sent)]#tmp_sentence[25])]
    m = []
    for i in range(0, len(s_m)):
        if s_m[i][1] < e_m[i][0]:
            m.append((s_m[i], e_m[i]))
        else:
            for j in range(i+1, len(e_m)):
                if s_m[i][1] < e_m[j][0]:
                    m.append((s_m[i], e_m[j]))
                    break;                   

    if len(m) > 0:
        for i in range(0, len(m)):
            if i == 0:
                tmp_sent1.append(input_sent[:m[i][0][0]])
            else :
                tmp_sent1.append(input_sent[m[(i-1)][1][1]:m[i][0][0]])                        
            tmp_sent2.append(input_sent[m[i][0][1]:m[i][1][0]])
        tmp_sent1.append(input_sent[m[-1][1][1]:])    
        return ' '.join(tmp_sent1 + tmp_sent2)
    else:
        return input_sent
    '''

In [9]:
step0_ptn= '[\'\‘\’]'
step1_ptn= '[\u00a0\u3000①②③④⑤⑥⑦⑧⑨⑩』◦※→®↑↓‣★▶■△◇◆▲○●\{\}\[\]\/?,+;:‧·ᆞ…》ⓒ|*~`\""“”!^_<>@\#&\\\=\'\n]'     
step2_ptn= '[\.]' 
article_sent = []
article_ner_l1, article_ner_l2 = [], []

for a in range(0, tmp_article.shape[0]):
    tmp_a = re.sub(pattern=step0_ptn, repl='\"', string=tmp_article.iloc[a])#.replace('\'‘’', '\"')
    tmp_a = re.sub(pattern=step1_ptn, repl='', string=tmp_a)
    tmp_sentence = kss.split_sentences(tmp_a)
    tmp_sentence = [re.sub(pattern=step2_ptn, repl='', string=s) for s in tmp_sentence]
    tmp_sentence = [parentheses_(s) for s in tmp_sentence] 

    ner_sent, ner_tag_l1, ner_tag_l2 = [], [], []
    pos_set = ref_dic['명칭'].values.tolist()

    for sent in range(0, len(tmp_sentence)):
        tmp_ner = []
        tmp_sent = re.sub(pattern=' +', repl=' ', string=tmp_sentence[sent])
        sent_split = tmp_sent.split(' ')
        tmp_sent_n, tmp_sent_a = divmod(len(sent_split), 30)
        tmp_mecab_pos = [mecab.pos(w) for w in sent_split]
        #tmp_ner = tmp_ner = [t for t in tmp_ner.split(' ') if len(t) >0]#tmp_ner.split(' ')
        tmp_ner_l1 = ['O'] * len(sent_split)#len(tmp_ner)
        tmp_ner_l2 = ['O'] * len(sent_split)#len(tmp_ner)

        for k in range(0 ,len(sent_split)):    
            m_pos = [i for i, tmp_me in enumerate(tmp_mecab_pos[k]) if tmp_me[1] == 'NNP']
            if len(m_pos) > 0:
                for l in range(0, len(m_pos)):#tmp_mecab_pos[4][m_pos[0]][0]
                    if tmp_mecab_pos[k][m_pos[l]][0] in pos_set:
                        ner_ind = pos_set.index(tmp_mecab_pos[k][m_pos[l]][0])                    
                        tmp_ner_l1[k] = ref_dic['대분류'].loc[ner_ind]
                        tmp_ner_l2[k] = ref_dic['중분류'].loc[ner_ind]
        ner_sent.append((' ').join(sent_split))
        ner_tag_l1.append((' ').join(tmp_ner_l1))
        ner_tag_l2.append((' ').join(tmp_ner_l2))
    article_sent.extend(ner_sent)
    article_ner_l1.extend(ner_tag_l1)
    article_ner_l2.extend(ner_tag_l2)

In [15]:
len(article_sent)

2991

In [16]:
label_ner_l1_ = [x for a_ner in article_ner_l1 for x in a_ner.split(' ') if x != 'O' and len(x) > 0 ]
label_ner_l2_ = [x for a_ner in article_ner_l2 for x in a_ner.split(' ') if x != 'O' and len(x) > 0]
tmp_label_ner_l1 = list(set(label_ner_l1_))
tmp_label_ner_l2 = list(set(label_ner_l2_))
tmp_label_ner_l1.sort()
tmp_label_ner_l2.sort()

In [17]:
label_ner_l1 = ['UNK']# + tmp_label_ner_l1
label_ner_l2 = ['UNK']# + tmp_label_ner_l2
for i in tmp_label_ner_l1:
    label_ner_l1.extend([i+'-B', i+'-I'])
for i in tmp_label_ner_l2:
    label_ner_l2.extend([i+'-B', i+'-I'])

In [18]:
# Train 
'''
with open('LGES_sent_210804.dta', 'wb') as a_sent:
    pickle.dump(article_sent, a_sent)
with open('LGES_sent_ner_level1_210804.dta', 'wb') as a_sent_l1:
    pickle.dump(article_ner_l1, a_sent_l1)   
with open('LGES_label_l1_210804.dta', 'wb') as a_label1:
    pickle.dump(label_ner_l1, a_label1)
'''
# Test 
with open('Test_LGES_sent_210804.dta', 'wb') as a_sent:
    pickle.dump(article_sent, a_sent)
with open('Test_LGES_sent_ner_level1_210804.dta', 'wb') as a_sent_l1:
    pickle.dump(article_ner_l1, a_sent_l1)   
with open('Test_LGES_label_l1_210804.dta', 'wb') as a_label1:
    pickle.dump(label_ner_l1, a_label1)

In [14]:
label_ner_l1

['UNK',
 '기술-B',
 '기술-I',
 '사건/사고-B',
 '사건/사고-I',
 '서비스-B',
 '서비스-I',
 '소재-B',
 '소재-I',
 '스펙-B',
 '스펙-I',
 '이벤트-B',
 '이벤트-I',
 '제품-B',
 '제품-I',
 '트렌드-B',
 '트렌드-I']

In [18]:
print('총 %d 개 문장'%len(article_ner_l1))
article_ner_l1[0]

총 605785 개 문장


'O O O O O O O O O O O O O O O O O O O'

In [19]:
tmp_total_sent = ' '.join(artice_ner_l1)

NameError: name 'artice_ner_l1' is not defined